## LLM 도메인 예측
### 타겟별 Noisy dataset을 예시로 넣고, 주제를 예측하라고 입력

In [5]:
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [6]:
# 사용할 LLM 모델 설정
llm = Ollama(model="gemma2")

/tmp/ipykernel_589628/3434335749.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma2")


In [11]:
import pandas as pd
from typing import Dict, List
import json
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     '''당신은 손상된 기사 제목들을 종합적으로 분석하여 전체 기사들의 공통된 도메인을 예측하는 어시스턴트입니다.
     주어지는 모든 기사 제목들을 함께 고려하여, 이 기사들이 속한 하나의 도메인을 판단하세요.
     도메인은 한국어이며, 한 단어로 표현해주세요.
     모든 기사 제목들에서 발견되는 주요 키워드들을 추출하세요.
     
     도메인 예시:
     - 정치
     - 북한
     - 경제
     - 산업
     - 사회
     - 전국
     - 세계
     - 문화
     - 건강
     - 연예
     - 스포츠
     
     다음 정보를 제공하세요:
     1. 모든 기사들에서 추출된 주요 키워드들을 하나의 리스트로
     2. 기사들의 공통된 도메인
     3. 도메인 예측 근거
     '''),
    ("human", 
     '''다음 기사 제목들을 분석하여 하나의 도메인으로 분류해주세요:
     
     기사들: {examples}
     
     {format_instructions}
     ''')
])

def create_unified_domain_classifier(llm):
    """통합 도메인 분류기 체인 생성"""
    chain = prompt | llm | parser
    
    def classify_domain(examples: List[str]) -> Dict:
        """여러 기사 제목들을 하나의 도메인으로 분류"""
        response = chain.invoke({
            "examples": "\n".join(examples),
            "format_instructions": """출력은 다음 JSON 형식이어야 합니다:
            {
                "keywords": ["키워드1", "키워드2", ...],
                "domain": "예측된_도메인",
                "reasoning": "도메인 예측 근거"
            }"""
        })
        return response
    
    return classify_domain

In [12]:
classifier = create_domain_classifier(llm)

examples = [
    "m 김정) 자주통일 새,?r열1나가야1보",
    "靑$김기식 논j:정면돌파 태세9선관위 '권해석 결과가 A수",
    "-용9 與 최고위원 출마…난 親~민계 당 위해 Kz",
    "지방서 새누-당 ,q 동v 촉구 이어져d일부는 7참 X대",
    "朴대통' 한 北비핵N 우선1칙따라 공조강화해야종1",
    "I기 나누는 1종석 비서실장과*한병i 정무T석",
    "文대통령n청년일자? 0술방망이 없ae하나씩 만들어가E",
    "되돌아본 한0 정-이A2C대 총~ 선거구 가J스로 지각:정",
    "유U 北제재 북한 5월 당대회서 경제`과V과시 [건xU가나",
    "단Z 朴 뇌2> 핵심B퍼즐 나왔다…i성에 최순실 5원-요구",
    "환영사$하는 문 대x령",
    "국정원장q*병우 직보 ?혹 국장,감찰 조사$중속보",
    "與텃밭/부산 Z변0더G주 4곳이{ 앞서",
    "[ '대통령재벌 총수 u공& 면담 경위 수사종b",
    "北0대! 김정*2향한 성경쟁 무대된l릴레이r토론"
]

results = classifier(examples)
print(json.dumps(results, indent=2, ensure_ascii=False))

{
  "주요 키워드": [
    "김정은",
    "문재인",
    "북한",
    "반도",
    "대통령",
    "경제",
    "사회",
    "핵무기",
    "미국",
    "전쟁",
    "의원",
    "정치"
  ],
  "공통된 도메인": "정치",
  "도메인 예측 근거": "제목에서 김정은, 문재인, 북한, 대통령 등 정치적 인물과 주제가 자주 등장하며, 경제, 사회, 핵무기 등 관련된 문제들 또한 언급되어 있습니다."
}


In [ ]:
classifier = create_domain_classifier(llm)

examples = [
    "땅 파= 코l나 격리시설 탈출한 외국인 청_서 VS",
    "모멘트 학교 급식U4칸막이lx곡",
    "초등학생들 경북도청 |들`",
    "한9%^6주택브랜드 공작소 론칭",
    "김영란법 때문에E애플 t이폰 초청70못받7 r국 언.",
    "부7교육청 D식종&자 등 7개 0. 육공무직 19}o 8발",
    "3주 사회서비스F 1급 사무/장 등 u부 직원 q명 공q",
    "코리아^레길 민간추진협의회 2<일 출u식",
    ")보통신산업진흥원 ~#7후보 3.수 압축",
    "0용 공고 보는XK직자들",
    "영상 -랑V 유학 포기할까요…한국 유학9G &6금 n상에 충격",
    "신#계A선호텔 국립전통예술고f*생에3장학금 R원",
    ",시y 9z방송편}인협회5김동연 부총리 초청 토론회",
    "금감원 이어 공정위까지…6부기관[사칭B악h메1 기승",
    "확진 속출F부산 사하구B모든 학교 이틀간 RV수업3q7교 대3",
    "충북.e7D년제 대학 @록금 H21만원…전국 평균 웃돌아",
    "黃 단식중단w도 패스트$랙|대치 @속…靑감찰의혹 공L 가열",
    "9살 학대 )동 4층P베란다 난간to3 넘어 목숨 건9맨발 _출",
    "오q지라이프{임원들 6사주y매각;보도는 사실무b",
    "항소심2선고 출석하는 이%택"
]

results = classifier(examples)
print(json.dumps(results, indent=2, ensure_ascii=False))

{
  "keywords": [
    "학교",
    "교육청",
    "부산",
    "경북",
    "학생",
    "공무직",
    "단식",
    "동",
    "사회서비스",
    "지역",
    "주택",
    "금융",
    "기업",
    "임원",
    "법",
    "경제",
    "국립전통예술고등학교",
    "유학",
    "민간추진협의회",
    "출식",
    "보도",
    "방송편자협회",
    "토론회",
    "사칭",
    "대학",
    "학원",
    "임금",
    "학교폭력",
    "청소년",
    "건강",
    "국가기관",
    "시정",
    "일제",
    "고용",
    "사회",
    "문화"
  ],
  "domain": "사회",
  "reason": "주요 키워드 분석 결과, 학교, 교육청, 부산, 경북 등 지역 사회 관련 용어와 공무직, 단식, 사회서비스, 지역 주택, 금융 기업 등 사회 문제와 연결된 키워드가 많이 발견됩니다. 또한 청소년, 건강, 국가기관과 같은 키워드도 포함되어 사회 전체에 대한 다양한 측면을 담고 있는 것으로 판단되었습니다."
}


### 특수문자 비율의 threshold를 0.2로 했을 때의 예시

In [16]:
noisy_df = pd.read_csv('basic_noisy.csv')
noisy_df = noisy_df.drop(columns=['Unnamed: 0', 'target_name', 're_text'])

# 각 target별 개수와 비율 구하기
target_counts = noisy_df['target'].value_counts()
target_ratios = noisy_df['target'].value_counts(normalize=True)

# 결과 출력
result_df = pd.DataFrame({
    'Count': target_counts,
    'Ratio': target_ratios
})

print(result_df)

        Count     Ratio
target                 
3          24  0.214286
6          20  0.178571
5          19  0.169643
0          15  0.133929
2          15  0.133929
1          13  0.116071
4           6  0.053571


In [18]:
df_0 = noisy_df[noisy_df['target']==0]
list_0 = df_0['text'].to_list()

In [19]:
classifier = create_domain_classifier(llm)

examples = list_0

results = classifier(examples)
print(json.dumps(results, indent=2, ensure_ascii=False))

{
  "keywords": [
    "화천",
    "눈썰매",
    "축제",
    "연극",
    "산",
    "호우",
    "경보",
    "목전",
    "롯데백화점",
    "예지",
    "독립 선언문"
  ],
  "domain": "문화",
  "reason": "제공된 기사 제목들을 분석해 보면 축제, 연극, 산과 관련된 내용이 눈에 띄며, 예지와 독립선언문 등 문화적 사건에 대한 기사도 포함되어 있습니다. 따라서 이러한 키워드들의 공통점으로 '문화' 도메인으로 분류했습니다."
}


In [21]:
classifier = create_domain_classifier(llm)

df_1 = noisy_df[noisy_df['target']==1]
list_1 = df_1['text'].to_list()
examples = list_1

results = classifier(examples)
print(json.dumps(results, indent=2, ensure_ascii=False))

{
  "keywords": [
    "농구",
    "야구",
    "축구",
    "프로야구",
    "대표팀",
    "한국",
    "개막",
    "선수",
    "경기",
    "승리",
    "시즌"
  ],
  "domain": "스포츠",
  "reason": "제공된 기사 제목들은 대부분 농구, 야구, 축구 등 다양한 스포츠 분야의 경기 개막, 선수, 시합 결과 등을 다루고 있습니다. 따라서 이들 기사들의 공통적인 주제는 '스포츠'입니다."
}


In [22]:
classifier = create_domain_classifier(llm)

df_2 = noisy_df[noisy_df['target']==2]
list_2 = df_2['text'].to_list()
examples = list_2

results = classifier(examples)
print(json.dumps(results, indent=2, ensure_ascii=False))

{
  "keywords": [
    "김기식",
    "청년",
    "문재인",
    "朴대통령",
    "북한",
    "최순실",
    "지방선거",
    "국정원",
    "경제",
    "핵심",
    "의외",
    "변동",
    "고위원",
    "수사",
    "토론"
  ],
  "domain": "정치",
  "reason": "기사 제목에서 다양한 정치적 키워드가 나타납니다. 대표적으로 '김기식 논란', '朴대통령', '문재인', '지방선거', '북한', '최순실', '국정원' 등이 언급되어 있습니다. 또한, '수사', '토론', '의외', '변동' 등의 키워드는 정치 현상에 대한 분석과 논의를 나타냅니다."
}


In [23]:
classifier = create_domain_classifier(llm)

df_3 = noisy_df[noisy_df['target']==3]
list_3 = df_3['text'].to_list()
examples = list_3

results = classifier(examples)
print(json.dumps(results, indent=2, ensure_ascii=False))

{
  "keywords": [
    "학교",
    "청소년",
    "주택",
    "교육",
    "정부",
    "여행",
    "사회",
    "금융",
    "사업",
    "뉴스",
    "인구",
    "학생",
    "경제",
    "법률",
    "오보",
    "동네",
    "공무원"
  ],
  "domain": "사회",
  "reason": "기사 제목에서 다양한 사회 현상에 대한 뉴스와 정보를 확인할 수 있습니다. 학교, 교육, 정부 정책, 경제, 법률, 사회 문제 등이 포함되어 있어 사회 분야를 대표하는 도메인으로 판단됩니다."
}


In [25]:
classifier = create_domain_classifier(llm)

df_5 = noisy_df[noisy_df['target']==5]
list_5 = df_5['text'].to_list()
examples = list_5

results = classifier(examples)
print(json.dumps(results, indent=2, ensure_ascii=False))

{
  "keywords": [
    "기업",
    "경제",
    "중소기업",
    "조합",
    "공동주택",
    "건설",
    "인건비",
    "증시",
    "가격지수",
    "해운",
    "금융",
    "자산",
    "생산",
    "제조업",
    "합병",
    "선임",
    "기본계약",
    "취득",
    "도시",
    "부동산",
    "출국금지",
    "브랜드"
  ],
  "domain": "경제",
  "reason": "제공된 기사 제목에서 '중소기업', '증시', '가격지수', '인건비', '합병', '생산', '금융', '부동산' 등 경제 분야와 직접적인 관련이 있는 키워드들이 주로 나타납니다. 또한, '해운', '경기', '중국'과 같은 주요 산업의 상황을 보여주는 단어들도 포함되어 있어 경제 전체를 다루고 있음을 시사합니다."
}


In [26]:
classifier = create_domain_classifier(llm)

df_6 = noisy_df[noisy_df['target']==6]
list_6 = df_6['text'].to_list()
examples = list_6

results = classifier(examples)
print(json.dumps(results, indent=2, ensure_ascii=False))

{
  "주요 키워드": [
    "중국",
    "일본",
    "북한",
    "프랑스",
    "미국",
    "아랍",
    "수단",
    "터키",
    "필리핀",
    "영국",
    "유엔",
    "브렉시트",
    "시위",
    "군부",
    "정치",
    "외교",
    "핵",
    "폭발",
    "사망",
    "전쟁",
    "경제",
    "투자",
    "스포츠",
    "교육"
  ],
  "도메인": "세계",
  "근거": "기사 제목에서 다양한 국가, 정치적 사건, 경제 이슈, 국제 관계 등 세계적인 주제들이 나타나고 있습니다."
}


### 특수문자 비율의 threshold를 0.25로 했을 때, target 4의 예시

In [27]:
noisy_df = pd.read_csv('basic_noisy_25.csv')
noisy_df = noisy_df.drop(columns=['Unnamed: 0', 'target_name', 're_text'])

# 각 target별 개수와 비율 구하기
target_counts = noisy_df['target'].value_counts()
target_ratios = noisy_df['target'].value_counts(normalize=True)

# 결과 출력
result_df = pd.DataFrame({
    'Count': target_counts,
    'Ratio': target_ratios
})

print(result_df)

        Count     Ratio
target                 
3          50  0.187266
6          46  0.172285
2          43  0.161049
5          39  0.146067
0          37  0.138577
1          30  0.112360
4          22  0.082397


In [28]:
classifier = create_domain_classifier(llm)

df_4 = noisy_df[noisy_df['target']==4]
list_4 = df_4['text'].to_list()
examples = list_4

results = classifier(examples)
print(json.dumps(results, indent=2, ensure_ascii=False))

{
  "keywords": [
    "스마트폰",
    "모바일",
    "로봇",
    "아이폰",
    "개발자",
    "게임",
    "기술",
    "인공지능",
    "디지털",
    "시큐리티",
    "전자파",
    "화이트 그래핀",
    "콘텐츠",
    "5G",
    "반도체",
    "수급",
    "패널",
    "고속인터넷",
    "목성",
    "챗봇",
    "데이터"
  ],
  "domain": "기술",
  "reason": "제공된 기사 제목들은 스마트폰, 로봇, 인공지능, 게임, 반도체 등 기술 분야를 다루고 있습니다. 또한 디지털 콘텐츠, 시큐리티, 전자파, 5G와 같은 키워드가 나타나 기술 발전과 관련된 주제들을 보여줍니다."
}


### 도메인이 예측한 {target: 도메인}

In [5]:
domains = ['문화', '스포츠', '정치', '사회', '기술', '경제', '세계']

target_to_domain = {
    '문화': 0,
    '스포츠': 1,
    '정치': 2,
    '사회': 3,
    '기술': 4,
    '경제': 5,
    '세계': 6
}

## LLM 키워드 뽑기

In [6]:
noisy_df = pd.read_csv('../train_noisy.csv')
noisy_df = noisy_df.drop(columns=['target_name', 're_text'])
noisy_df

,ID,text,target
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2
3,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6
4,ynat-v1_train_00006,프로야구~롯TKIAs광주 경기 y천취소,1
...,...,...,...
1582,ynat-v1_train_02787,13이 노바 라이n2·미J어패0 T3 10 결G상품% *,6
1583,ynat-v1_train_02788,남원소식 춘>X학%단+장Rn 모집,0
1584,ynat-v1_train_02789,"이총리,세4. H직 완fl지 못해E할 일 꽤;남아",2
1585,ynat-v1_train_02792,"높`X#E율…}BO Q""[/선수 몸값 상승 CAO",1


In [7]:
# 각 target별 개수와 비율 구하기
target_counts = noisy_df['target'].value_counts()
target_ratios = noisy_df['target'].value_counts(normalize=True)

# 결과 출력
result_df = pd.DataFrame({
    'Count': target_counts,
    'Ratio': target_ratios
})

print(result_df)

        Count     Ratio
target                 
6         229  0.144297
2         228  0.143667
5         227  0.143037
3         226  0.142407
4         226  0.142407
0         226  0.142407
1         225  0.141777


In [8]:
domain_to_target = {v: k for k, v in target_to_domain.items()}

# noisy_df의 target 열을 텍스트로 변환
noisy_df['target'] = noisy_df['target'].map(domain_to_target)
noisy_df

,ID,text,target
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,기술
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,사회
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",정치
3,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,세계
4,ynat-v1_train_00006,프로야구~롯TKIAs광주 경기 y천취소,스포츠
...,...,...,...
1582,ynat-v1_train_02787,13이 노바 라이n2·미J어패0 T3 10 결G상품% *,세계
1583,ynat-v1_train_02788,남원소식 춘>X학%단+장Rn 모집,문화
1584,ynat-v1_train_02789,"이총리,세4. H직 완fl지 못해E할 일 꽤;남아",정치
1585,ynat-v1_train_02792,"높`X#E율…}BO Q""[/선수 몸값 상승 CAO",스포츠


In [9]:
import pandas as pd
from collections import defaultdict
from typing import Dict, List
import json
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

def create_domain_keyword_extractor(llm):
    """도메인별 키워드 추출기 생성"""
    
    # JSON 출력 파서 초기화
    parser = JsonOutputParser()
    
    # 프롬프트 템플릿 설정
    prompt = ChatPromptTemplate.from_messages([
        ("system", 
         '''당신은 여러 텍스트를 분석하여 해당 도메인을 대표하는 핵심 키워드를 추출하는 전문가입니다.
         주어진 텍스트들은 모두 같은 도메인에 속하며, 텍스트에 노이즈가 첨가되어있습니다.
         이 텍스트들을 분석하여 다음을 만족하는 키워드를 추출해주세요.
         
         1. 해당 도메인을 가장 잘 나타내는 핵심 키워드 50개를 추출해주세요
         2. 키워드는 해당 도메인을 다른 도메인과 구분 짓는 특징적인 단어여야 합니다
         3. 일반적인 단어보다는 도메인 특화된 전문 용어나 고유명사를 선호합니다
         4. 추출된 키워드는 중요도 순으로 정렬해주세요
         
         현재 분석 중인 도메인: {domain}
         '''),
        ("human", 
         '''다음 텍스트들을 분석하여 도메인 특화된 키워드를 추출해주세요:
         
         텍스트들: {examples}
         
         {format_instructions}
         ''')
    ])
    
    # 체인 생성
    chain = prompt | llm | parser
    
    def extract_keywords(domain: str, texts: List[str]) -> List[str]:
        """특정 도메인의 텍스트들에서 키워드 추출"""
        response = chain.invoke({
            "domain": domain,
            "examples": "\n".join(texts),
            "format_instructions": """출력은 다음 JSON 형식이어야 합니다:
            {
                "keywords": ["키워드1", "키워드2", ...]
            }"""
        })
        return response["keywords"]
    
    return extract_keywords

def process_domains_and_save(noisy_df: pd.DataFrame, llm, output_path: str = "domain_keywords.csv"):
    """모든 도메인의 텍스트를 처리하고 결과를 CSV로 저장"""
    
    # 키워드 추출기 생성
    extractor = create_domain_keyword_extractor(llm)
    
    # 도메인별 키워드 저장할 딕셔너리
    domain_keywords = defaultdict(list)
    
    # 각 도메인별로 처리
    for domain in noisy_df['target'].unique():
        # 해당 도메인의 텍스트만 추출
        domain_texts = noisy_df[noisy_df['target'] == domain]['text'].tolist()
        
        # 키워드 추출
        keywords = extractor(domain, domain_texts)
        
        # 결과 저장
        domain_keywords[domain] = keywords
    
    # 결과를 데이터프레임으로 변환
    # 가장 긴 키워드 리스트 길이 찾기
    max_keywords = max(len(keywords) for keywords in domain_keywords.values())
    
    # 모든 리스트를 같은 길이로 맞추기 (짧은 리스트는 None으로 패딩)
    for domain in domain_keywords:
        domain_keywords[domain].extend([None] * (max_keywords - len(domain_keywords[domain])))
    
    # 데이터프레임 생성 및 저장
    result_df = pd.DataFrame(domain_keywords)
    result_df.to_csv(output_path, index=False, encoding='utf-8')
    
    return result_df

In [39]:
# 결과 생성 및 저장
result_df = process_domains_and_save(noisy_df, llm, "domain_keywords_word.csv")

# 결과 확인
print(result_df.head())

            기술      사회    정치               세계   스포츠     경제      문화
0         삼성전자    노동실태  국정원장      중국 해외투자 안전도  프로농구     경기      만화
1  5GX MES 플랫폼     교육부  朴대통령     3중 대사관 조기 게양    축구    부동산   독립선언문
2         스마트폰  어린이 안전    북한    한아세 베트남 관계 악화   김재현     금리     사진전
3        디스플레이    한국노총   김정은  페루 대통령 정치 공정 제안   손흥민     외환   사진미술관
4         지문인식    사회복지   무수성        이스라엘 정보당국   류현진  인플레이션  전국리기대회


### 텍스트마다 키워드 뽑고, 중복 제거하기

In [10]:
import pandas as pd
from typing import Dict, List, Set
import json
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

def create_text_keyword_extractor(llm):
    """개별 텍스트에서 키워드를 추출하는 함수 생성"""
    
    # JSON 출력 파서 초기화
    parser = JsonOutputParser()
    
    # 프롬프트 템플릿 설정
    prompt = ChatPromptTemplate.from_messages([
        ("system", 
         '''당신은 텍스트를 분석하여 가장 중요한 키워드를 추출하는 전문가입니다.
         주어진 텍스트는 {target} 도메인에 속합니다. 이 텍스트를 분석하여:
         
         1. 텍스트의 핵심 내용을 가장 잘 나타내는 1-2개의 키워드만 추출해주세요
         2. 키워드는 해당 도메인을 특징짓는 전문 용어나 고유명사여야 합니다
         3. 일반적인 단어는 제외하고, 도메인 특화된 단어만 선택해주세요
         4. 적절한 키워드가 없다면 빈 리스트를 반환해도 됩니다
         '''),
        ("human", 
         '''다음 텍스트에서 {target} 도메인의 핵심 키워드를 1-2개만 추출해주세요:
         
         텍스트: {text}
         
         {format_instructions}
         ''')
    ])
    
    chain = prompt | llm | parser
    
    def extract_keywords(target: str, text: str) -> List[str]:
        """단일 텍스트에서 키워드 추출"""
        response = chain.invoke({
            "target": target,
            "text": text,
            "format_instructions": """출력은 다음 JSON 형식이어야 합니다:
            {
                "keywords": ["키워드1"] 또는 ["키워드1", "키워드2"] 또는 []
            }"""
        })
        return response["keywords"]
    
    return extract_keywords

def process_target_texts(df_target: pd.DataFrame, target: str, llm) -> Dict:
    """특정 타겟의 모든 텍스트를 처리하고 결과 저장"""
    
    # 키워드 추출기 생성
    extractor = create_text_keyword_extractor(llm)
    
    # 모든 키워드를 저장할 세트 (중복 자동 제거)
    all_keywords = set()
    
    # 각 텍스트별로 처리
    for idx, text in enumerate(df_target['text']):
        try:
            # 키워드 추출
            keywords = extractor(target, text)
            # 추출된 키워드가 있으면 세트에 추가
            all_keywords.update(keywords)
            
            # 진행상황 출력 (선택사항)
            if (idx + 1) % 10 == 0:
                print(f"{target} 도메인: {idx + 1}/{len(df_target)} 텍스트 처리 완료")
                
        except Exception as e:
            print(f"텍스트 처리 중 오류 발생: {e}")
            continue
    
    # 결과 딕셔너리 생성
    result = {
        "target": target,
        "keywords": list(all_keywords)  # 중복이 제거된 키워드 리스트
    }
    
    return result

def save_keywords(result: Dict, output_dir: str = "keywords"):
    """추출된 키워드를 JSON 파일로 저장"""
    import os
    
    # 저장 디렉토리 생성
    os.makedirs(output_dir, exist_ok=True)
    
    # JSON 파일로 저장
    target = result["target"]
    output_path = os.path.join(output_dir, f"{target}_keywords.json")
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=2)

def combine_all_keywords(output_dir: str, output_path: str = "all_keywords.csv"):
    """저장된 모든 JSON 파일을 읽어서 하나의 DataFrame으로 통합"""
    import os
    import glob
    
    # 모든 JSON 파일 읽기
    json_files = glob.glob(os.path.join(output_dir, "*_keywords.json"))
    
    # 결과를 저장할 딕셔너리
    all_keywords = {}
    
    # 각 JSON 파일 처리
    for json_file in json_files:
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
            all_keywords[data['target']] = data['keywords']
    
    # DataFrame 생성
    # 가장 긴 키워드 리스트 찾기
    max_len = max(len(keywords) for keywords in all_keywords.values())
    
    # 길이 맞추기
    for target in all_keywords:
        all_keywords[target].extend([None] * (max_len - len(all_keywords[target])))
    
    # DataFrame 생성 및 저장
    df = pd.DataFrame(all_keywords)
    df.to_csv(output_path, index=False, encoding='utf-8')
    
    return df

In [12]:
# 각 타겟별로 처리
targets = ['문화', '스포츠', '정치', '사회', '기술', '경제', '세계']
for target in targets:
    # 타겟별 데이터 추출
    df_target = noisy_df[noisy_df['target'] == target]
    
    # 키워드 추출 및 중복 제거
    result = process_target_texts(df_target, target, llm)
    
    # 결과 저장
    save_keywords(result)
    print(f"\n{target} 도메인 처리 완료")
    print(f"추출된 고유 키워드 수: {len(result['keywords'])}")

# 모든 결과 통합
final_df = combine_all_keywords('keywords', 'all_domain_keywords.csv')
print(final_df.head())

문화 도메인: 10/226 텍스트 처리 완료
문화 도메인: 20/226 텍스트 처리 완료
문화 도메인: 30/226 텍스트 처리 완료
문화 도메인: 40/226 텍스트 처리 완료
문화 도메인: 50/226 텍스트 처리 완료
문화 도메인: 60/226 텍스트 처리 완료
문화 도메인: 70/226 텍스트 처리 완료
문화 도메인: 80/226 텍스트 처리 완료
문화 도메인: 90/226 텍스트 처리 완료
문화 도메인: 100/226 텍스트 처리 완료
문화 도메인: 110/226 텍스트 처리 완료
문화 도메인: 120/226 텍스트 처리 완료
문화 도메인: 130/226 텍스트 처리 완료
문화 도메인: 140/226 텍스트 처리 완료
문화 도메인: 150/226 텍스트 처리 완료
문화 도메인: 160/226 텍스트 처리 완료
텍스트 처리 중 오류 발생: Invalid json output: {
    "keywords": []
} 


주어진 텍스트는 '명말청6 7기[?국7사mz들은 3IUSb아남@p' 와 같이 의미를 파악할 수 없는 형태로 제시되어 있습니다. 따라서 문화 도메인의 키워드를 추출하기 어렵습니다.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE
문화 도메인: 170/226 텍스트 처리 완료
문화 도메인: 180/226 텍스트 처리 완료
문화 도메인: 190/226 텍스트 처리 완료
문화 도메인: 200/226 텍스트 처리 완료
문화 도메인: 210/226 텍스트 처리 완료
문화 도메인: 220/226 텍스트 처리 완료
텍스트 처리 중 오류 발생: 'NoneType' object is not subscriptable

문화 도메인 처리 완료
추출된 고유 키워드 수: 70
스포츠 도메인: 10/225 텍스트 처리 완료
스포츠 도메인: 20/225 텍스트 처리 완료
스포츠 도메인: 30/225 텍스트 처리 완료
스포

In [13]:
final_df

,기술,사회,문화,스포츠,경제,세계,정치
0,py체,대외고 통학차,스포X카,득점왕,기업수익,대선,국회
1,뉴욕 필름 페스,미세먼지,교향곡,MLB,배당,협약 이행,책 추진
2,아이펀팩`리,교황청,포퓰리즘,올림픽,경기침체,해상N력,소통 강화
3,왓x,항소심,불교,바셀로나,합병,정상회의,국회사무처
4,AI 업지능,파장,모z르트w,바르사 빌바오,부품주,美4中Z군사적 충돌,올림픽
...,...,...,...,...,...,...,...
139,None,거제 E0 출입제한,None,도전,None,None,None
140,None,지방자치단체,None,두산,None,None,None
141,None,D식종&자,None,프로야구,None,None,None
142,None,민간추진협의회,None,루카쿠,None,None,None


## Noisy text 복원

In [14]:
import pandas as pd
from typing import Dict, List
import json
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

def create_text_restorer(llm):
    """텍스트 복원기 생성"""
    
    # JSON 출력 파서 초기화
    parser = JsonOutputParser()
    
    # 프롬프트 템플릿 설정
    prompt = ChatPromptTemplate.from_messages([
        ("system", 
         '''당신은 노이즈가 낀 텍스트를 원래의 뉴스 기사 제목으로 복원하는 전문가입니다.
         주어진 텍스트는 {target} 도메인의 뉴스 기사 제목이었으나 노이즈가 끼었습니다.
         
         복원 규칙:
         1. 노이즈가 낀 텍스트를 보고 원래 의미를 파악할 수 있다면, 해당 의미를 살려 복원해주세요
         2. 텍스트가 너무 손상되어 의미 파악이 어렵다면, 제공된 키워드 리스트에서 적절한 키워드를 선택해 
            {target} 도메인에 맞는 새로운 뉴스 기사 제목을 생성해주세요
         3. 복원된 텍스트의 길이는 원본 텍스트의 글자 수와 최대한 비슷하게 맞춰주세요
            (원본 길이: {original_length}자)
         4. 복원 방식(복원/생성)을 명시해주세요
         '''),
        ("human", 
         '''다음 정보를 바탕으로 텍스트를 복원해주세요:
         
         도메인: {target}
         노이즈가 낀 텍스트: {noisy_text}
         참고 키워드 리스트: {keywords}
         
         {format_instructions}
         ''')
    ])
    
    chain = prompt | llm | parser
    
    def restore_text(target: str, noisy_text: str, keywords: List[str]) -> Dict:
        """텍스트 복원 실행"""
        response = chain.invoke({
            "target": target,
            "noisy_text": noisy_text,
            "keywords": ", ".join(keywords),
            "original_length": len(noisy_text),
            "format_instructions": """출력은 다음 JSON 형식이어야 합니다:
            {
                "restored_text": "복원된 텍스트",
                "method": "복원" 또는 "생성"
            }"""
        })
        return response
    
    return restore_text

def process_dataframe(noisy_df: pd.DataFrame, keyword_df: pd.DataFrame, llm) -> pd.DataFrame:
    """전체 데이터프레임 처리"""
    
    # 결과를 저장할 새로운 데이터프레임 생성
    result_df = noisy_df.copy()
    result_df['re_text'] = ''
    result_df['restoration_method'] = ''
    
    # 텍스트 복원기 생성
    restorer = create_text_restorer(llm)
    
    # 각 행별로 처리
    for idx, row in result_df.iterrows():
        try:
            # 해당 타겟의 키워드 리스트 가져오기
            target_keywords = keyword_df[row['target']].dropna().tolist()
            
            # 텍스트 복원
            result = restorer(
                target=row['target'],
                noisy_text=row['text'],
                keywords=target_keywords
            )
            
            # 결과 저장
            result_df.at[idx, 're_text'] = result['restored_text']
            result_df.at[idx, 'restoration_method'] = result['method']
            
            # 진행상황 출력 (100개마다)
            if (idx + 1) % 100 == 0:
                print(f"{idx + 1}/{len(result_df)} 처리 완료")
                
        except Exception as e:
            print(f"ID {row['ID']} 처리 중 오류 발생: {e}")
            result_df.at[idx, 're_text'] = row['text']  # 오류 시 원본 텍스트 유지
            result_df.at[idx, 'restoration_method'] = 'error'
            continue
    
    return result_df

def save_results(df: pd.DataFrame, output_path: str = "restored_texts.csv"):
    """결과를 CSV 파일로 저장"""
    df.to_csv(output_path, index=False, encoding='utf-8')

In [15]:
# 전체 데이터프레임 처리
restored_df = process_dataframe(noisy_df, final_df, llm)

# 결과 저장
save_results(restored_df, "restored_news_titles.csv")

# 결과 확인
print("\n복원 방법 통계:")
print(restored_df['restoration_method'].value_counts())

# 샘플 결과 확인
print("\n복원 예시:")
samples = restored_df.sample(5)
for _, row in samples.iterrows():
    print(f"\n원본: {row['text']}")
    print(f"복원: {row['re_text']}")
    print(f"방법: {row['restoration_method']}")

ID ynat-v1_train_00046 처리 중 오류 발생: Invalid json output: ```json
{
"restored_text": "5자농 R4은행 삼[m명 꺾직",
"method": "복원"
}
``` 


**해설:**

주어진 노이즈가 낀 텍스트는 스포츠 관련 뉴스 제목 형태로 보이며, 일부 단어들이 잘못 표기되어 있습니다.  '5자농 R4은행 삼[m명 꺾직' 을 통해 '5자농 R4은행 삼성 명 꺾직'으로 복원할 수 있을 것으로 추정됩니다.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE
100/1587 처리 완료
200/1587 처리 완료
ID ynat-v1_train_00474 처리 중 오류 발생: 'NoneType' object is not subscriptable
300/1587 처리 완료
ID ynat-v1_train_00566 처리 중 오류 발생: Invalid json output: ```json
{
"restored_text": "한국 교향악단 오케스트라 공연",
"method": "복원"
}
``` 



**설명:**

노이즈가 끼어있는 ' [간 @기와 ;외X 음k5quA타 3q' 에서  '교향악단', '오케스트라' 와 같은 키워드를 추출하여 원래 의미로 가정하여 복원했습니다.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE
ID ynat-v1_train_00579 처리 중 오류 발생: Invalid json output: ```json
{
"restored_text": "영국, 레지스탕스 혼란에 C z?獨(佛…*y한 입장 제Q해"[합",
"method": "복원"
}
``` 


**요약:**

제공된 노이즈가 낀 텍스트는  '영국의

## 도메인 + 키워드 이용 데이터 증강

### 사이먼이 준 키워드 이용

In [17]:
import pandas as pd 
from ast import literal_eval

df = pd.read_csv('train_noisy_keyword.csv')

# 문자열을 리스트로 변환
def str_to_list(x):
    try:
        if type(x) == str:
        	return literal_eval(x)
        elif type(x) == list:
        	return x
    except: #해당 값이 null값이거나 오류가 있을 때, None을 return 하기
        return None

df['keyword'] = df['keyword'].apply(lambda x: str_to_list(x))
df

,topic,keyword
0,문화,"[초9세 학생들, 고3 학생, 소프트웨어, 시상, 베트남, 물놀이, 높이지 않고야 ..."
1,스포츠,"[검찰, 마틴 스나이더, 지역별 경쟁, 인생, 베트남, 경쟁 치열, MVP, 비위,..."
2,정치,"[면담, 유능한 경제팀, DMZ, 프랑스 방문, 더불어민주당, 미국, 바람, 예산 ..."
3,사회,"[방송편성위원회, 검찰, 론칭, 견학, 사법부, 수능, 철도사업, 상가, 전기차, ..."
4,기술,"[ICT업계, 5큐빅 박스, 검찰, 액세서리, 미국 우주인, 뇌종성 암, 문보안, ..."
5,경제,"[연구 지원, 주택시장, 검찰, 베트남, 해커스제과식품, 공사, 2분기, 코오롱하늘..."
6,세계,"[검찰, 어획량 감소, 42%, 화산재, 공사, 아사드, 이슈, 北 무수단 발사, ..."


In [24]:
def create_keyword_extractor(llm):
    """키워드 리스트에서 핵심 키워드 50개를 추출하는 함수 생성"""
    
    # JSON 출력 파서 초기화
    parser = JsonOutputParser()
    
    # 프롬프트 템플릿 설정
    prompt = ChatPromptTemplate.from_messages([
        ("system", 
         '''당신은 주어진 키워드 리스트 중에서, 해당 도메인을 대표하는 핵심 키워드 50개를 추출하는 전문가입니다.
         주어진 키워드는 {target} 도메인에 속합니다. 주어진 키워드를 분석하여:
         
         1. 해당 도메인을 가장 잘 나타내는 핵심 키워드 50개를 추출해주세요
         2. 키워드는 해당 도메인을 다른 도메인과 구분 짓는 특징적인 단어여야 합니다
         3. 일반적인 단어보다는 도메인 특화된 전문 용어나 고유명사를 선호합니다
         4. 적절한 키워드가 없다면 50개보다 적게 반환해도 됩니다
         5. 추출된 키워드는 중요도 순으로 정렬해주세요
         
         도메인 별 주요 키워드 예시 및 유의점
         - 문화: 건강정보, 자동차/시승기, 도로/교통, 여행/레저, 음식/맛집, 패션/뷰티, 공연/전시, 책, 만화/웹툰, 학술/문화재, 종교, 날씨, 생활문화 일반과 관련된 키워드를 정확히 뽑아주세요. 특히, 관련된 특정 브랜드, 장소, 음식 이름, 인물 등이 나타난다면 키워드로 추출하세요.
         - 스포츠: 야구, 축구, 농구, 배구, 골프와 같은 스포츠 종목과 관련된 키워드를 뽑아주세요. 특히, 팀 이름, 선수 이름, 경기 기록(예: 득점, 승패), 리그 명칭이 있다면 이를 키워드로 추출해주세요.
         - 정치: 대통령실/총리실, 국회/정당, 외교, 국방 관련 키워드를 뽑아주세요. 중요한 법안, 외교 정책, 정부 조직 이름, 주요 인물 이름, 그리고 정치적 사건이나 논쟁이 나타난다면 이를 구체적으로 키워드로 추출해주세요. 특히, 사회 도메인과 구분되는 정치 분야만 추출하도록 유의하세요.
         - 사회: 사건/사고, 법원/검찰, 교육, 복지/노동, 환경, 여성/아동, 재외동포, 다문화와 관련된 키워드를 뽑아주세요. 중요한 사건 이름, 기관 명칭, 인물 이름이나 조직, 법률과 판결 관련 키워드가 포함될 경우, 정확하게 추출하여 키워드로 표시해주세요. 특히, 정치와 문화 도메인과 헷갈리지 않도록 관련성이 높은 사회 분야 키워드만 포함해주세요.
         - 기술: 과학 기술과 IT 기술 전반을 포괄하는 키워드를 뽑아주세요. 특히 주요 IT 기업 이름(예: 구글, 애플)이나 제품, 서비스, 기술 용어가 등장할 경우 정확히 추출해주세요.
         - 경제: 경제/정책, 금융, 부동산, 취업/창업, 소비자와 관련된 키워드를 정확히 뽑아주세요. 주식, 펀드, 선물 등의 금융 용어나 경제 지표, 가격 변동(상승, 하락) 관련 키워드가 나타나면 이를 구체적으로 추출해주세요. 특히 회사 이름, 산업 이름 등도 주요 키워드로 포함해주세요.
         - 세계: 미국/북미, 중국, 일본, 아시아/호주, 유럽, 중남미, 중동/아프리카, 국제기구와 관련된 키워드를 뽑아주세요. 국가 이름, 도시 이름, 국제기구 이름, 세계적인 정세와 관련된 용어가 나타난다면 이를 키워드로 구체적으로 추출해주세요.
         
         현재 분석중인 도메인: {target}
         '''),
        ("human", 
         '''다음 키워드 리스트에서 {target} 도메인의 핵심 키워드 50개를 추출해주세요:
         
         키워드 리스트: {keywords}
         
         {format_instructions}
         ''')
    ])
    
    chain = prompt | llm | parser
    
    def extract_keywords(target: str, noisy_text: str, keywords: List[str]) -> Dict:
        """키워드 추출 실행"""
        response = chain.invoke({
            "target": target,
            "keywords": ", ".join(keywords),
            "format_instructions": """출력은 다음 JSON 형식이어야 합니다:
            {
                "target": "target"
                "keywords": [키워드 1, 키워드 2, .., 키워드 50]
            }"""
        })
        return response
    
    return extract_keywords


def process_dataframe(df: pd.DataFrame, llm) -> pd.DataFrame:
    """전체 데이터프레임에서 각 도메인별 핵심 키워드를 추출하는 함수
    
    Args:
        df (pd.DataFrame): topic과 keyword 컬럼을 포함한 데이터프레임
        llm: LangChain 호환 LLM 객체
    
    Returns:
        pd.DataFrame: 핵심 키워드가 추가된 데이터프레임
    """
    
    # 결과를 저장할 새로운 데이터프레임 생성
    result_df = df.copy()
    result_df['key_keywords'] = None
    
    # 키워드 추출기 생성
    extractor = create_keyword_extractor(llm)
    
    # 각 행별로 처리
    for idx, row in result_df.iterrows():
        try:
            # keyword 컬럼이 이미 리스트 형태이므로 직접 사용
            keywords = row['keyword']
            
            # 키워드 추출 실행
            result = extractor(
                target=row['topic'],
                noisy_text="",  # 사용하지 않는 파라미터이므로 빈 문자열 전달
                keywords=keywords
            )
            
            # 결과 저장
            result_df.at[idx, 'key_keywords'] = result['keywords']
            
            # 진행상황 출력
            print(f"'{row['topic']}' 도메인 처리 완료 ({idx + 1}/{len(result_df)})")
                
        except Exception as e:
            print(f"'{row['topic']}' 도메인 처리 중 오류 발생: {e}")
            result_df.at[idx, 'key_keywords'] = []  # 오류 시 빈 리스트로 설정
            continue
    
    return result_df


def save_results(df: pd.DataFrame, output_path: str = "extracted_noisy_keyword.csv"):
    """결과를 CSV 파일로 저장"""
    df.to_csv(output_path, index=False, encoding='utf-8')

In [25]:
result_df = process_dataframe(df, llm)

'문화' 도메인 처리 완료 (1/7)
'스포츠' 도메인 처리 완료 (2/7)
'정치' 도메인 처리 완료 (3/7)
'사회' 도메인 처리 완료 (4/7)
'기술' 도메인 처리 완료 (5/7)
'경제' 도메인 처리 완료 (6/7)
'세계' 도메인 처리 완료 (7/7)


In [26]:
save_results(result_df, "extracted_noisy_keyword.csv")

In [29]:
result_df = result_df.drop(columns=['keyword'])
result_df.to_csv('extracted_noisy_keyword.csv')

In [30]:
result_df

,topic,key_keywords
0,문화,"[춘향단, 호텔, 발레, 사진, 일본, 삼성차, 소리길, 국민, 대화, 행진, 발표..."
1,스포츠,"[표팀, 양민, 삼성사, 고려고, 득점, 수비신공, 성적, 스타들, 올림피아드, 바..."
2,정치,"[위성, 정당, 국회사무처, 대선 후보 출정식, 협력, 프레임, 지원, 김정은, 북..."
3,사회,"[체 추진, 등교, 공개채, 경제 협력, 서검사, 한국노총, 터키, 과학기술연구소,..."
4,기술,"[력, 정책, 발표, 언팩, 물사진, 유출, 지원, 둥근 화면, 출시 예정, 보안,..."
5,경제,"[반도체 공장, 주택담보대출, 영업이익, 신도시, 증여, 33층 빌딩, LG에너지솔..."
6,세계,"[정치권, 화웨이, 흑자, 대만, 군사적 충돌, 터키, 조사 착수, 노딜 브렉시트,..."


### 데이터 증강

In [33]:
import pandas as pd
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

# 요청 사항 설정
req = "도메인을 명확하게 나타내는 뉴스 기사 제목을, 30글자가 넘지 않도록 도메인별로 30개씩 생성해 주세요."

# JSONOutputParser 초기화
parser = JsonOutputParser()

# 도메인별 요구사항
domain_requirements = """
- 문화: 건강, 자동차 리뷰, 교통 상황, 여행지 추천, 맛집 탐방, 패션 트렌드, 공연 정보, 신간 도서, 만화/웹툰, 학술/문화재, 종교 행사, 기상 소식 등의 주제를 포함해야 하며, 제목은 생활 속 유용한 정보나 최신 문화 트렌드를 반영해야 합니다. 특정 활동이나 장소가 포함되면 더 좋습니다.
- 스포츠: 야구, 축구, 농구, 배구, 골프와 관련된 스포츠 소식이나 팀 이름, 선수 이름, 경기 결과를 포함하면 좋습니다. 제목은 해당 종목의 주요 경기 소식, 팀이나 선수의 승패, 리그 경기 일정을 반영하는 내용이어야 합니다. 팬들에게 흥미를 줄 수 있도록 특정 팀, 선수, 경기 기록이 포함되면 좋습니다.
- 정치: 대통령실, 국회, 정당 정책, 외교 관계, 국방 관련 주제로 정치적인 이슈를 다루면 좋습니다. 사회 분야와 헷갈리지 않도록 법안, 외교 협력, 정부 정책 등의 정치적 의제를 명확히 드러내고, 국내외 정치적 흐름이나 중요한 정부 발표가 포함된 제목이어야 합니다.
- 사회: 사건/사고, 법원 판결, 교육 정책, 복지 서비스, 노동 문제, 환경 이슈, 여성과 아동 문제, 다문화 관련 주제를 다루면 좋습니다. 정치나 문화 분야와 헷갈리지 않도록, 사회적인 현안이나 공공의 문제를 강조하고, 특히 법원이나 검찰 관련 사안, 주요 사건을 포함한 제목이 좋습니다.
- 기술: 유명 IT 기업 이름이나 신기술 관련 용어가 포함되어 기술의 발전과 변화를 잘 드러내는 제목을 만들어주세요. 최신 과학 기술, IT 산업의 발전, 기술 기업의 신제품 발표, 혁신적인 기술 트렌드와 관련되면 좋습니다. 기술의 응용이나 영향을 언급하면 더욱 좋습니다.
- 경제: 경제 정책, 금융 시장, 부동산, 취업/창업, 소비자 관련 정보를 담기면 좋습니다. 주식, 펀드, 금리, 부동산 가격 상승/하락 등 경제 변동을 보여주는 내용을 포함하고, 특히 투자나 기업의 경제적 성장, 경기 흐름을 반영하는 제목이 좋습니다.
- 세계: 미국, 중국, 일본, 유럽 등 국가 이름이나 국제기구와 관련된 글로벌 소식을 담기면 좋습니다. 세계적인 정세 변화를 반영하고, 국가 간 관계, 국제 협력, 글로벌 이슈와 같은 주제를 다루며, 특정 지역이나 국가의 중요 소식이 포함된 제목을 만들어주세요.
"""

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages([
    ("system",
     '''당신은 특정 도메인의 기사 제목을 생성하는 어시스턴트입니다.
     다음 조건을 기반으로 기사 제목을 생성해 주세요:

     - 주어진 도메인 "{{target}}"을 명확하게 나타내는 한문장 내외의 적절한 뉴스 기사 제목을 30개 생성하세요.
     - 제목은 반드시 JSON 배열 형식으로 반환하고, 도메인을 "target" 키로, 생성한 제목을 "text" 키로 표시합니다.

     도메인별 요구사항:
     {domain_requirements}

     주의: 응답은 JSON 배열 형식으로만 반환해야 하며, 그 외의 텍스트는 포함하지 마세요. 
     반드시 JSON 형식으로만 반환하고, 배열의 각 항목은 {{"target": "{{target}}", "text": "<생성한 제목>"}} 형식이어야 합니다.'''),
    ("user",
     "#Format: {format_instructions}\n\n#Requirements: {req}\n\n#Domain: {target}")
])

# 프롬프트, 모델, 파서를 연결하는 체인 생성
chain = prompt | llm | parser

def generate_domain_titles(domains: List[str]) -> pd.DataFrame:
    """
    여러 도메인에 대한 뉴스 제목을 생성하고 DataFrame으로 반환하는 함수
    
    Args:
        domains (List[str]): 뉴스 도메인 리스트
    
    Returns:
        pd.DataFrame: target(도메인)과 text(제목) 컬럼을 가진 DataFrame
    """
    data = []
    
    for domain in domains:
        try:
            print(f"\n{domain} 도메인의 뉴스 제목 생성 중...")
            response = chain.invoke({
                "req": req,
                "target": domain,
                "domain_requirements": domain_requirements,
                "format_instructions": parser.get_format_instructions()
            })
            
            # response가 None인지 확인
            if response is None:
                print("오류 발생: chain.invoke()에서 None이 반환되었습니다.")
            elif isinstance(response, list):  
                data.extend(response)
            else:
                print("올바르지 않은 JSON 응답:", response)

        except Exception as e:
            print(f"{domain} 도메인에서 예외 발생: {str(e)}")

    return pd.DataFrame(data)

In [40]:
domains = ['문화'] # '문화', '스포츠', '정치', '사회', '기술', '경제', '세계'

# 뉴스 제목 생성
df = generate_domain_titles(domains)


문화 도메인의 뉴스 제목 생성 중...
문화 도메인에서 예외 발생: Invalid json output: [
{"target": "문화", "text": "‘강변 한가운데’ 힙스터들의 새로운 취미"},
{"target": "문화", "text": "올해의 인기 여행지: 프랑스 알프스의 눈꽃"},
{"target": "문화", "text": "신간 '인생 최고' - 명상과 행복 팁 전파"},
{"target": "문화", "text": "전망 좋은 서울 카페, 데이트 추천 곳! Top3"},
{"target": "문화", "text": "'이혼소녀', 한국 웹툰 인기"},
{"target": "문화", "text": "여름철 냉면 대결: 지역 소개와 레시피!"},
{"target": "문화", "text": "고급 백화점 패션쇼, 트렌디한 스타일 노출"},
{"target": "문화", "text": "'뮤지컬 라인', 사랑과 환상의 흥미진진 공연"},
{"target": "문화", "text": "BTS 콘서트 티켓 대박! 팬덤 열광"},
{"target": "문화", "text": "전국 대형 백화점, 여행 상품 할인 행사 진행"},
{"target": "문화", "text": "2023년 인기 드라마, 시청률 급상승!"},
{"target": "문화", "text": "'김치찌개' 세계로 확산! 한국 문화 확산?"},
{"target": "문화", "text": "한국의 전통 음악, 가요와의 조화"},
{"target": "문화", "text": "전국 맛집 대잔치, 지역 특색 다양"},
{"target": "문화", "text": "K팝 아이돌, 국제 무대 위에서 활약!"},
{"target": "문화", "text": "독립영화 '시민의 자유', 비평가들의 호소"},
{"target": "문화", "text": "한국 전통 공예품, 디자인 트렌드 반영"},
{"target": "문화", "text": "'호텔 델루나' 시즌2 기대감 높아지다!"},
{

In [35]:
# 각 target별 개수와 비율 구하기
target_counts = df['target'].value_counts()
target_ratios = df['target'].value_counts(normalize=True)

# 결과 출력
result_df = pd.DataFrame({
    'Count': target_counts,
    'Ratio': target_ratios
})

print(result_df)

        Count     Ratio
target                 
스포츠        29  0.460317
사회         28  0.444444
문화          6  0.095238


In [36]:
domains = ['문화', '스포츠', '정치', '사회', '기술', '경제', '세계']

target_to_domain = {
    '문화': 0,
    '스포츠': 1,
    '정치': 2,
    '사회': 3,
    '기술': 4,
    '경제': 5,
    '세계': 6
}

df['target'] = df['target'].map(target_to_domain)

In [37]:
base = pd.read_csv('short_news_titles.csv')
auged_df = pd.concat([base, df], ignore_index=True)
auged_df = auged_df.drop(columns=['Unnamed: 0'])

In [38]:
# 각 target별 개수와 비율 구하기
target_counts = auged_df['target'].value_counts()
target_ratios = auged_df['target'].value_counts(normalize=True)

# 결과 출력
result_df = pd.DataFrame({
    'Count': target_counts,
    'Ratio': target_ratios
})

print(result_df)

        Count     Ratio
target                 
1         287  0.148936
4         285  0.147898
2         283  0.146860
3         282  0.146341
6         279  0.144785
5         266  0.138038
0         245  0.127141


In [39]:
auged_df.to_csv('short_news_titles.csv')

### target 정수화

In [125]:
domains = ['문화', '스포츠', '정치', '사회', '기술', '경제', '세계']

target_to_domain = {
    '문화': 0,
    '스포츠': 1,
    '정치': 2,
    '사회': 3,
    '기술': 4,
    '경제': 5,
    '세계': 6
}

In [126]:
# auged_df target 열을 정수로 변환
auged_df['target'] = auged_df['target'].map(target_to_domain)
auged_df

,target,text
0,0,"**'나의 아티스트, 나만의 색깔' - 서울 시립 미술관 신규 전시 기대감 UP!**"
1,0,**여름 맞이 '냉면 vs 비빔밥' 맛집 대결! 당신은 어떤 선택?**
2,0,"'스타일 변신', 패션 트렌드 집중 분석 - 2023년 여름 인기 소품 대방출!"
3,0,"**'인생영화, 다시 한번!' 오리지널 OST 반복 감상 이유?**"
4,0,**독서 습관 형성! 새롭게 출간된 베스트셀러 도서 추천 목록**
...,...,...
820,3,"국내 우주 개발 사업 궤도 올라, '우주 선진국' 목표 시기 조정"
821,3,"'여성 고용 불평등', 해결책 모색 위해 정부 간담회 개최"
822,3,"아동학대 사건 증가, '사회적 지원 시스템' 강화 필요성 제기"
823,3,"중앙은행, 금리 인상로 통해 inflation 통제 위해 노력"


In [128]:
# 특수 기호 제거: !, ?, *, -를 지움
auged_df['text'] = auged_df['text'].str.replace(r'[!?*-]', '', regex=True)

# 변경된 DataFrame을 CSV 파일로 저장 (필요한 경우)
auged_df.to_csv('cleaned_generated.csv', index=False)

In [ ]:
restored_df = restored_df.drop(columns=['text'])
restored_df = restored_df.rename(columns={'re_text': 'text'})
restored_df['target'] = restored_df['target'].map(target_to_domain)
restored_df

,ID,target,text,restoration_method
0,ynat-v1_train_00000,4,"KT AI 서비스 이용기간 2년 단축, Q분종 U2보",복원
1,ynat-v1_train_00001,3,K찰국 회장 노조 통합협상,복원
2,ynat-v1_train_00002,2,"김정은, 북미 대화 정면돌파 필요하나?",복원
3,ynat-v1_train_00004,6,미대선 앞두고 R2프랑스 단 발 비해 감시 강화,복원
4,ynat-v1_train_00006,1,프로야구 롯데 KIA 광주 경기 취소,복원
...,...,...,...,...
1582,ynat-v1_train_02787,6,13일 노바 라이2·미군 합동演習 이루어진다.,복원
1583,ynat-v1_train_02788,0,남원소식춘향단 모집,복원
1584,ynat-v1_train_02789,2,"이총리, 국회 직 무기대상책 추진해야",복원
1585,ynat-v1_train_02792,1,"높`X#E율…}BO Q""[/선수 몸값 상승 CAO",error


In [134]:
restored_df = restored_df[restored_df['restoration_method']!='error']

In [135]:
gemma_made = pd.concat([auged_df, restored_df], ignore_index=True)
gemma_made

,target,text,ID,restoration_method
0,0,"'나의 아티스트, 나만의 색깔' 서울 시립 미술관 신규 전시 기대감 UP",NaN,NaN
1,0,여름 맞이 '냉면 vs 비빔밥' 맛집 대결 당신은 어떤 선택,NaN,NaN
2,0,"'스타일 변신', 패션 트렌드 집중 분석 2023년 여름 인기 소품 대방출",NaN,NaN
3,0,"'인생영화, 다시 한번' 오리지널 OST 반복 감상 이유",NaN,NaN
4,0,독서 습관 형성 새롭게 출간된 베스트셀러 도서 추천 목록,NaN,NaN
...,...,...,...,...
2390,3,텅 빈 교실 선생님 외롭게,ynat-v1_train_02786,복원
2391,6,13일 노바 라이2·미군 합동演習 이루어진다.,ynat-v1_train_02787,복원
2392,0,남원소식춘향단 모집,ynat-v1_train_02788,복원
2393,2,"이총리, 국회 직 무기대상책 추진해야",ynat-v1_train_02789,복원


In [136]:
gemma_made.to_csv('gemma_made.csv')

## 리라벨링

In [ ]:
from langchain.llms import Ollama
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import ChatPromptTemplate
import pandas as pd
from typing import List, Dict
import json
import time

def classify_news_articles(df: pd.DataFrame, examples: str, llm: Ollama) -> pd.DataFrame:

    # JSON 출력 파서 초기화
    parser = JsonOutputParser()

    # 프롬프트 템플릿 설정
    prompt = ChatPromptTemplate.from_messages([
        ("system", 
         '''당신은 손상된 기사 제목들을 종합적으로 분석하여 전체 기사들의 공통된 도메인을 예측하고, 주어지는 텍스트를 해당 도메인으로 분류하는 어시스턴트입니다.
         아래 제시되는 주제 번호(1-7)의 예시 기사들을 분석하여, 각 주제 번호가 나타내는 도메인을 파악하고 새로운 기사를 분류하세요.
         
         분류 과정:
         1. 각 주제 번호의 예시 기사들을 분석하여 공통된 특징과 키워드를 파악
         2. 파악된 특징을 바탕으로 해당 주제 번호의 도메인을 결정
         3. 새로운 기사를 가장 적합한 주제 번호로 분류
         4. 분류 근거가 되는 핵심 키워드 추출
         
         출력 형식은 반드시 아래의 JSON 형식을 따라야 합니다: 
         {{"text": "입력된 기사 제목", "topic": "분류된 주제 번호(1-7)", "keywords": ["분류 근거가 된 키워드들"]}}
         다른 설명이나 부가적인 텍스트는 포함하지 마세요.
         '''),
        ("human", 
         '''다음 예시 기사 제목들을 분석하여 각 주제 번호의 도메인을 파악하고, 새로운 기사를 분류해주세요:
         
         {examples}
         
         분류할 텍스트: {text}
         ''')
    ])

    # 결과를 저장할 리스트 초기화
    results: List[Dict] = []
    
    # 전체 데이터 수
    total_articles = len(df)
    
    # 각 텍스트 분류 실행
    for idx, row in df.iterrows():
        try:
            # 진행상황 출력
            print(f"Processing article {idx+1}/{total_articles}...")
            
            # 프롬프트 포맷팅
            formatted_prompt = prompt.format(
                examples=examples,
                text=row['text']
            )
            
            # Ollama로 응답 받기
            response = llm.invoke(formatted_prompt)
            
            try:
                # JSON 파싱 시도
                result = json.loads(response)
                results.append(result)
            except json.JSONDecodeError:
                print(f"Error parsing JSON for article {idx+1}. Raw response: {response}")
                results.append({
                    "text": row['text'],
                    "topic": "error",
                    "keywords": []
                })
            
            # API 호출 간격 조절
            time.sleep(0.5)
            
        except Exception as e:
            print(f"Error processing article {idx+1}: {str(e)}")
            results.append({
                "text": row['text'],
                "topic": "error",
                "keywords": []
            })
    
    # 결과를 데이터프레임으로 변환
    results_df = pd.DataFrame(results)
    
    # 원본 데이터프레임과 결과 병합
    final_df = pd.concat([df, results_df[['topic', 'keywords']]], axis=1)
    
    return final_df

In [9]:
examples = """
주제 번호 0:
- 구z 사이로 또 한 해가( 충남 해넘' 명소 북Z
- 불교U서 육식] 언제]터 금지됐을까
- 갯벌도 얼려버린^한파
- 천은사 Q장료 폐지 업무9P
- 100만 체험= 목전0 둔 화천|산천어축3
- Y십3 빛본 화0q비니u  I 9쳐야 한단 사실 일찍J깨달아
- 화천산천어`제2즐거운 눈썰>
- 거창u제연극제를 강탈{려1 합니t
- 미국을 사로잡_ 예지 올여름 단독D}한공연
- :용}이 140년전 옥중에서 쓴g독립선5문 5 nW
- #계의w숙제 소수 b미의 수수께끼_풀릴까
- 제주도A산간 오H 3Q 호우0의보→호우경보
- "신간M마르크g""의와|한국의 인문학"
- 책7빌A기 쉬워^V…3종시 S공도서관 10곳|자$ 공동 활용

주제 번호 1:
- ~ 전 앞둔eW재 감98T이 부K되지만  4로 승부수
- ,7안컵 선V단장에 7영일$축구7회 부회장6우Bh위해 지원
- 최0웅-현대캐피탈 감j 이승원 Lu력 대단하다
- 프As구L수원 5정원 감독 16강 $는 길에 방심=*없다
- 美 독립리그에서 프로야구#.초로x타자가 =루'도루
- 한국 야구 아H안게임 금0달 향해 본격 \d
- 데이비스 4"점$뉴올리언s 8T전7=에 보스턴 4연{ 저지
- 박9서l '트남 스즈키& 1차전 라오스에9r0 대`
- 남북Z일농t경기대회 방북단 군용기로 내일4평양"방문
- 전광인4없어도 한국전L FTV4컵서 현:캐피탈 격파종합
- ~학농구리- 8일 고려대중앙대 경pt 개막
- 수호신 리베라 사상 R 4장일o로 MUB 명예의 q_ 입성
- 최NA 멀티히트 l& 3출Z q약…탬파베이 4연패 탈출

주제 번호 2:
- "m 김정) 자주통일 새,?r열1나가야1보",
- "靑$김기식 논j:정면돌파 태세9선관위 '권해석 결과가 A수",
- "-용9 與 최고위원 출마…난 親~민계 당 위해 Kz",
- "지방서 새누-당 ,q 동v 촉구 이어져d일부는 7참 X대",
- "朴대통' 한 北비핵N 우선1칙따라 공조강화해야종1",
- "I기 나누는 1종석 비서실장과*한병i 정무T석",
- "文대통령n청년일자? 0술방망이 없ae하나씩 만들어가E",
- "되돌아본 한0 정-이A2C대 총~ 선거구 가J스로 지각:정",
- "유U 北제재 북한 5월 당대회서 경제`과V과시 [건xU가나",
- "단Z 朴 뇌2> 핵심B퍼즐 나왔다…i성에 최순실 5원-요구",
- "환영사$하는 문 대x령",
- "국정원장q*병우 직보 ?혹 국장,감찰 조사$중속보",
- "與텃밭/부산 Z변0더G주 4곳이{ 앞서",
- "[ '대통령재벌 총수 u공& 면담 경위 수사종b",
- "北0대! 김정*2향한 성경쟁 무대된l릴레이r토론"

주제 번호 3:
- "땅 파= 코l나 격리시설 탈출한 외국인 청_서 VS",
- "모멘트 학교 급식U4칸막이lx곡",
- "초등학생들 경북도청 |들`",
- "한9%^6주택브랜드 공작소 론칭",
- "김영란법 때문에E애플 t이폰 초청70못받7 r국 언.",
- "부7교육청 D식종&자 등 7개 0. 육공무직 19}o 8발",
- "3주 사회서비스F 1급 사무/장 등 u부 직원 q명 공q",
- "코리아^레길 민간추진협의회 2<일 출u식",
- ")보통신산업진흥원 ~#7후보 3.수 압축",
- "0용 공고 보는XK직자들",
- "영상 -랑V 유학 포기할까요…한국 유학9G &6금 n상에 충격",
- "신#계A선호텔 국립전통예술고f*생에3장학금 R원",
- ",시y 9z방송편}인협회5김동연 부총리 초청 토론회",
- "금감원 이어 공정위까지…6부기관[사칭B악h메1 기승",
- "확진 속출F부산 사하구B모든 학교 이틀간 RV수업3q7교 대3",
- "충북.e7D년제 대학 @록금 H21만원…전국 평균 웃돌아",
- "黃 단식중단w도 패스트$랙|대치 @속…靑감찰의혹 공L 가열",
- "9살 학대 )동 4층P베란다 난간to3 넘어 목숨 건9맨발 _출",
- "오q지라이프{임원들 6사주y매각;보도는 사실무b",
- "항소심2선고 출석하는 이%택"

주제 번호 4:
- 떼었S 붙였다 액세서리Y 스마C폰 업U레이드
- 해외로밍 m금폭탄 n동차단 더 빨$진다
- 스마트폰'b실 막는 스l트 트%커
- 삼성 빅스. 개발자데KS1B월 서울B 첫 +최
- ?0 V3 모바일 시큐리티 :50k T]로드 돌파
- 그라}리tA 디지털 콘텐츠를f상품*로 만드세0
- "넥슨 게;/버}진 개발사 아이펀팩`리에 전""적 투자"
- 화이트 그래핀 py체 나올)…대면적 합= 성공
- WSJ 스크린 5함 0란 갤Lb 폴드 조롱 동|상
- 온종일 차는 스s트!치 전자파i얼마나 나vh
- 베일HS싸인 탑I형 로봇2 평창 성화 e고 첫  a
- 면`세포의 배신…뇌종R 성/e돕는0면역6포 발V
- 게시판 배틀그$q드'여름맞P 경< 행사
- 갤노트7 \i 7 M 사장단회의…사장단 침w
- 국`계Y%58 수립에 {공지능 기v2도입한다
- 나사 목성의 L성 5로파서 2증기-발+q흔J 발z1보
- 손6) 대화 로봇 $드세요{네이)=챗봇 빌더 @개
- 아이폰X 수급 다소 호AyD기기간 1주 단g

주제 번호 5:
- 중소기업 JB 경기(망 부정적…인건비 올라|최대 애로
- 국토h 공동주택 관리 n우수0지에M서울 텐r힐 1단지
- 에스.유 255억k O스플7이 제조n비 J급계약
- 해운 위기?용r료 인하.합병으로 해결될 문l ;냐
- 특징주 메가B디(자회사 합병에 급W
- &년lN북협력기금 9d178억 S출H2016년보다 855억 많아
- 증시u6품 동양자산\V @이나 전환사채 펀5
- 철퇴 맞는 비트코인g중국서 출국금지y~국선7사기_ (소
- 그래픽 제조업 평균 가p률#p이
- 한진그룹&3@ 조wU 대한항공 총괄 부사장에 선임
- `1테오닉 독일 업체에L!플란트i스템 공3 기본계약
- g신 S_ 사외:사에 권력기관 출신 9사들 K천
- 오션a6지 중& 자회사 지분 2억~O 취득
- 여성등반가 rG8?소기 흡입력z로 33층 빌딩 올랐*
- 10대그8 v장사^토지가액 73조…현대차 최고 땅C자
- 게시판 신W금; 더 뱅커 선정 대한민|31등 DR 브랜드
- .양피5에프 h5억원i{모 플랜트설비 납품 계약

주제 번호 6:
- ;가 프리다a 사상 이해 안돼…벌@ =신 주멕시7J美대사
- 바레인 아랍의H봄 xU부 시위 시아파\지도자에C종신형
- 푸틴 .리아 철군 계획 "사드x 사전 협의0아사드' R8
- 3중XQ4대사관 3안먼 시a 3K주V에 조기 게양
- 프랑스 노딜 브렉시0 대>_본u화
- 터키 화2 우세 vb워 쿠르드 ?|T민간인 피o 커져종합2보
- 필리핀에 불법 수출" g나다산A쓰레I^ 되돌아간다
- 군부에 굴복 안 해…수단 야권114N?g파업"예
- it룽의 반지 국내 제작6초W…명랑한 바그너 입[
- o국 초등학교 3부% 투Y 교육 시킨-
- 프랑스 이란에 핵OW 의무 %0L삼가라
- 일본  출6이u아 15일 n국…22일 고베 입단식
- v시H 축출후 수단 군부정권 이끄N 아우w는
- 中인민일보.美 중국/내정간a은d비열한 행위 Y비난
- 아프리카 남수a서 부04간 폭g사태로 11명L사망
- 유엔 w.아4공습으로 1[일간 민간인 100명 이상 yZ
- t7정상회의 균? 심각L한 목소리 &낸다
- 英장관 e명7공개 반기…브렉시트 8의 없)면 연기I지지
"""

In [ ]:
# Ollama 모델 설정
llm = Ollama(model="gemma2")

# 예시 데이터프레임 생성
df = pd.read_csv('../train_not_noisy.csv')
sample_df = df[['text']]

# 분류 실행
result_df = classify_news_articles(sample_df, examples, llm)

# 결과 출력
print("\nClassification Results:")
print(result_df)

## 리라벨링 ver.2

In [9]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import pandas as pd
import json
from typing import List
import asyncio
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import nest_asyncio

# 주피터 노트북에서 비동기 실행을 위한 설정
nest_asyncio.apply()

class TextClassifier:
    def __init__(self):
        self.llm = Ollama(
            model="gemma2",
            callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
        )
        
        self.prompt_template = PromptTemplate(
            input_variables=["examples", "text"],
            template="""당신은 주어지는 text를 특정 도메인으로 분류 해야 합니다. 이때 도메인은 특정지어 도메인은 특정지어 주어지지 않고, '주제 번호'로 주어집니다.
            각 주제 번호(0~6)에 대한 예시가 주어지면, 각 주제 번호에 예시로 주어지는 기사들의 공통적인 도메인을 예측하고, 주제 번호를 도메인과 연결지어야 합니다.
            그 후에, 주어지는 텍스트들을 도메인별로 분류합니다. 즉, 아래의 순서를 따릅니다:
            1. 예시로 주어지는 기사들을 분석하여 예시들이 속하는 공통된 도메인을 예측한다.
            도메인 예시:
            - 정치
            - 북한
            - 경제
            - 산업
            - 사회
            - 전국
            - 세계
            - 문화
            - 건강
            - 연예
            - 스포츠
            
            2. 각 주제 번호를 예측한 도메인과 연결짓는다.
            3. 주어지는 텍스트들을 예측한 도메인들 중 가장 들어맞는 하나로 분류한다.
            4. 출력형식은 반드시 아래의 JSON 형식을 따른다:
            {{"text": 입력된 기사 제목, "topic": 분류된 주제 번호(숫자만)}}
            
            다음 예시 기사 제목들을 분석하여 각 주제 번호의 도메인을 파악하고, 새로운 기사를 분류해주세요:
            
            {examples}
            
            분류할 텍스트: {text}
"""
        )
        
        self.chain = LLMChain(llm=self.llm, prompt=self.prompt_template)
    
    async def classify_text(self, text: str, examples: str) -> Dict:
        """단일 텍스트 분류"""
        response = await self.chain.arun(examples=examples, text=text)
        try:
            return json.loads(response)
        except json.JSONDecodeError:
            return {"text": text, "topic": -1}
    
    async def classify_texts(self, texts: pd.Series, examples: str) -> List[Dict]:
        """여러 텍스트 분류"""
        tasks = [self.classify_text(text, examples) for text in texts]
        return await asyncio.gather(*tasks)

async def process_texts(examples: str, texts: pd.Series):
    # 분류기 초기화 및 실행
    classifier = TextClassifier()
    results = await classifier.classify_texts(texts, examples)
    
    # 결과를 DataFrame으로 변환 및 저장
    results_df = pd.DataFrame(results)
    results_df.to_csv('classification_results.csv', index=False, encoding='utf-8-sig')
    print(f"Results saved to classification_results.csv")
    return results_df

In [10]:
# 사용 예시
examples = """
주제 번호 0의 예시:
- 7월 1일부터 도로 청소 의무화
- 제주도 산간에 3시간 동안 호우 경보 발령
- 과학의 숙제 소수의 수수께끼 풀릴까
- 용비어천가 140년 전 옥중에서 쓴 독립선언문 5문 5편
- 롯데백화점 완벽한 웨딩 행사 진행
- 거창제 연극제를 강탈하려 합니다
- 화천 산천어 축제 제2회 즐거운 눈썰
- 100만 체험객 목전에 둔 화천 산천어축제
- 갯벌도 얼어붙은 한파
- 구즈 사이로 또 한 해가 (충남 해넘이 명소 북제주)
- 스타필드 하남 개장 후 이틀 동안 3만 명 방문
- 동물도 생명권이 있다...소도 집에서 채식주의
- 과학의 숙제 소수의 수수께끼 풀릴까
- 46+통 5분식점 65곳 된 왕실다...서울 노포 39곳 지도에

주제 번호 1의 예시:
- 고려대학교 중앙도서관 개관 8일 기념 행사 개막
- "전광인 없어도 한국 남자 배구 대표팀, FVTV컵서 현대로템 격파 종합"
- 박 감독 '베트남 스즈키컵' 1차전 라오스에 대승
- 데이비스 점 뉴올리언스 8전 7에 보스턴 4연패 저지
- KB국민은행이 농아인 국가대표팀과 6년간 후원 계약을 체결했다
- 한국 야구 아시안게임 금메달 향해 본격 돌입
- 원주 문무대학교 물들로...강원 제비네레 내달 개막
- 美 독립리그에서 프로야구 초로 타자가 도루
- 최웅-현대캐피탈 감독 이승원 대단하다
- AFC 아시안컵 단장에 7명의 후보, 부회장 6명 선출 위해 지원
- 수호신 리베라 사상 최고의 4장타자로 MUB 명예의 전당 입성
- "축구 대표팀 감독, 귀화 선수에게 허가통지서 전달...대표팀 합류만 남았다"
- 최정의 멀티히트 3출루...탬파베이 4연패 탈출
- 7월 3일의 야구장 나들이...임준택 30일구 개 전 시구

주제 번호 2의 예시:
- "김정은 ""자주통일 새시대 열어나가야"
- 北조국통 민주화운동 전선 결성 0돌 기념 보고회 열어
- 선거 여론조사 결과 유출 논란
- 새누리 원내대표가 여소야대와 계파 문제를 언급
- "남북정상회담 준비가 생명, 남북 리허설 또 리허설"
- 북한 당대회 시작하면서 만약의 상황 대비
- "김정은 ""3차 평양 과학거리 설립 지시"
- "문재인 대통령, 독립운동 역사 공유하면 북한의 마식령 스키장도 가까운 것"
- "박근혜 대통령, 북핵은 턱밑 위협 해결 않으면 한미에 큰 피바람"
- 靑 김기식 논란 정면돌파 태세...선관위 '권한 해석 결과가 수
- 문재인 대통령, '평화 선언' 바람직하다는 데 공감대
- 여당 최고위원 출마...난 친박계 당 위해 헌신
- 北김정은 당 대회 함경도·황해도·강원도 대표로 추대
- 이 총리 나누는 1종석 비서실장과 한병도 정무 수석
- 주주의 이재명:징계 일단 유보...당 안팎 우려 제기(란 잠재종합2보)

주제 번호 3의 예시:
- 학교 석면 관리 규정 지키기 5년
- 땅 파 코로나 격리시설 탈출한 외국인 청부살인 시도 VS
- 모멘트 학교 급식 칸막이 설치 논란
- "한신공영, 6월 주택브랜드 '공소소' 론칭"
- 안경은 착용 안 되고...경찰관 공채 신체검사 기준 논란
- 게시 | 예독거노인 고치기 봉사활동
- "교육부 관계자 등 7개 부처 ""육성 기업직 19%"" 8월 발표"
- 종교시설 방역수칙 위반 사업자 증가...교황청 지침 존재한
- 검 이화대 교수 및 관계자 사무실·주택 4곳 압수수색 (0여명 + 1명) 색출 보도
- 3주 사회서비스 1급 사무/장 등 부장 직원 20명 공개채용
- 한국기자협회 창립 51년 기념
- 코리아레길 민간추진협의회 2일 출범식
- 거제 엑스포 월간 출입제한 동계 정비 3일 내 재개방
- 정보통신기획평가원 3차원장 후보 7명 압축
- 공공기관 채용 공고 보는 구직자들

주제 번호 4의 예시:
- 해외 로밍 요금 폭탄 동차단 더 빨라진다
- 스마트폰 '블루라이트' 막는 필름 출시
- 삼성 빅스비 개발자대회 11월 서울서 첫 개최
- 떼었다가 붙였다 액세서리 스마트폰 업그레이드
- V3 모바일 시큐리티 50주년 기념 로드 돌파
- 그라픽 디지털 콘텐츠를 상품으로 만드세요
- "넥슨, 게임 개발사 아이펀팩토리에 전적 투자"
- 온종일 차는 스마트폰! 전자파 얼마나 나한
- 베일에 싸인 탑형 로봇 2호 평창 성화 점화 첫 시도
- 면억세포의 배신...뇌종성 암 돕는 면역 요법 발표
- 위클리 스마트 스트레스엔 명상과 면역 강화엔 커피
- 게시판 배틀그라운드 여름맞이 행사
- "패널업계, V6 시장 침체해 벗어나기 위해 새로운 비즈니스 모델 찾는다"
- 한국 1위 고속인터넷 가입자...KT 가입자 10만
- 갤노트7 사장단회의...사장단 침목
- 나사 목성의 위성 5개에서 수증기 발견-발사 성공(종합)
- 아이폰X 수급 다소 호조 기간 1주 단축

주제 번호 5의 예시:
- 중소기업 경기 부진...인건비 올라 최대 애로
- 국토교통부 공동주택 관리 우수지에 서울 텐즈힐 1단지 선정
- 삼성전자 255억원 스플린터 제조비 계약
- 특징주 메가바이트(자회사 합병에 급유동성)
- "남북협력기금 9178억 지원, 2016년보다 855억 많아"
- 그 아파트 매매가격지수 변동률
- 철퇴 맞는 비트코인, 중국서 출국금지~국선 7사기 (소셜)
- 증시에서 동양자산운용의 전환사채 발행
- 그래픽 제조업 평균 가동률 증가
- 한진그룹 조양호 대한항공 총괄 부사장에 선임
- 테슬라 독일 업체에 플랜드시스템 공급 3년 기본계약
- 오션씨앤씨 중国 자회사 지분 2억 원 취득
- 10대 그룹이 장사하는 토지가액 73조...현대차 최고 땅 가진 기업
- 피 5억원 규모 플랜트설비 납품 계약

주제 번호 6의 예시:
- 美 미네소타 주지사 입구에서 2명 부상
- "반기문 유엔 사무총장 부부, 관저에 초대 만찬...2년전 약속 지켜"
- 英장관 명 공개 반기...브렉시트 8월 없으면 연기 지지
- G7 정상회의 균형 잡힌 목소리 낸다
- 유엔, 전쟁으로 1일간 민간인 100명 이상 사망
- 아프리카 남부에서 부룬디 간폭발 사고로 11명 사망
- "중국 인민일보, 미국의 중국 내정 간섭은 비열한 행위라며 비난"
- "우크라이나, 일본에 '마리오처럼 싸우겠다'며 투자 호소"
- 군부 축출 후 수단 군부정권 이끄는 오마르 알바시르는
- "일본 출신 선수 15일 입국, 22일 고베 입단식"
- 프랑스 이란에 핵 의무 삼가라
- 필리핀에 불법 수출된 쓰레기 되돌아간다
- 터키 화염이 우세, 쿠르드 민간인 피해 커져종합2보
- 프랑스 노딜 브렉시트 본화
- "푸틴, 리아가 철군 계획에 대해 ""사드 배치 사전 협의 없었다""고 아사드 언급"
- 가고 싶다 프리다 칼 사상 이해 안 돼...벌금 물려 신 주메시 미국 대사
"""